# List of Live Matches from cricbuzz website

https://www.cricbuzz.com/cricket-match/live-scores

# GET MATCH ID FOR LIVE MATCHES FROM BELOW OUTPUT

In [38]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://www.cricbuzz.com/cricket-match/live-scores"
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")
matches = soup.find_all("div", class_="cb-mtch-lst cb-col cb-col-100 cb-tms-itm")

for match in matches:
    title = match.find("h3").text.strip()
    print("Live Match Name : ",title)
    link_tag = match.find('a', class_='text-hvr-underline text-bold')
    if link_tag:
        relative_link = link_tag['href']
        full_link = "https://www.cricbuzz.com" + relative_link
        print("\nLink for this live match details :\n",full_link)
    else:
        print("Link not found")
    try:
        match_id = full_link.split("/")[4]
        print("\nMatch id is :",match_id)
    except IndexError:
        print("Could not extract match ID from:", full_link)
        continue

    status = match.find("div", class_="cb-text-live").text.strip() if match.find("div", class_="cb-text-live") else \
             match.find("div", class_="cb-text-complete").text.strip() if match.find("div", class_="cb-text-complete") else \
             "Status Unavailable"

    print(f"\n{status}")
    print("\n***********************************************************************************************************************************")
    print("***********************************************************************************************************************************")
    print("\n")

Live Match Name :  Netherlands vs United Arab Emirates,

Link for this live match details :
 https://www.cricbuzz.com/live-cricket-scores/117584/ned-vs-uae-62nd-match-icc-cricket-world-cup-league-two-2023-27

Match id is : 117584

United Arab Emirates opt to bowl

***********************************************************************************************************************************
***********************************************************************************************************************************


Live Match Name :  Thailand Women vs United Arab Emirates Women,

Link for this live match details :
 https://www.cricbuzz.com/live-cricket-scores/118583/thaiw-vs-uaew-6th-match-womens-t20i-quadrangular-series-in-thailand-2025

Match id is : 118583

Thailand Women won by 16 runs

***********************************************************************************************************************************
********************************************************

# Extracting Data For One Match Id

In [11]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

headers = {"User-Agent": "Mozilla/5.0"}
match_id = input("\nEnter Cricbuzz Match ID (e.g., 118583): ").strip()

title = "Match"
try:
    live_url = "https://www.cricbuzz.com/cricket-match/live-scores"
    response = requests.get(live_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    for link in soup.find_all("a", href=True):
        if f"/{match_id}/" in link["href"]:
            title = link.text.strip()
            break
except Exception:
    pass

api_url = f"https://www.cricbuzz.com/api/cricket-match/commentary/{match_id}"
res = requests.get(api_url, headers=headers)

if res.status_code != 200:
    print("Failed to fetch commentary. Check match ID.")
    exit()

data = res.json()
rows = []

for item in data.get("commentaryList", []):

    raw_comment = item.get("commText") or item.get("commentaryText") or ""
    formats = item.get("commentaryFormats", {})
    bold_formats = formats.get("bold", {})
    format_ids = bold_formats.get("formatId", [])
    format_values = bold_formats.get("formatValue", [])

    if format_ids and format_values:
        for fmt_id, fmt_val in zip(format_ids, format_values):
            raw_comment = raw_comment.replace(fmt_id, fmt_val)

    rows.append({
        "over": item.get("overNumber"),
        "ball": item.get("ballNbr"),
        "event": item.get("event"),
        "commentary": raw_comment,
        "batting_team": item.get("batTeamName"),
        "batsman": item.get("batsmanStriker", {}).get("batName"),
        "batsman_runs": item.get("batsmanStriker", {}).get("batRuns"),
        "bowler": item.get("bowlerStriker", {}).get("bowlName"),
        "timestamp": item.get("timestamp")
    })

df = pd.DataFrame(rows)
filename = f"match_{match_id}_commentary.csv"
df.to_csv(filename, index=False)
print(f"\nCommentary for '{title}' saved to: {filename}")

print("\n Top 10 rows of Commentary data :")
print(df.head(10))



Enter Cricbuzz Match ID (e.g., 118583): 117584

Commentary for 'NED vs UAE - Live' saved to: match_117584_commentary.csv

 Top 10 rows of Commentary data :
   over  ball       event                                         commentary  \
0  20.6   126  over-break                Junaid Siddique to Edwards, no run    
1  20.5   125        NONE                Junaid Siddique to Edwards, no run    
2  20.4   124        NONE                Junaid Siddique to Edwards, no run    
3  20.3   123        NONE                Junaid Siddique to Edwards, no run    
4  20.2   122        NONE               Junaid Siddique to Max ODowd, 1 run    
5  20.1   121        NONE                 Junaid Siddique to Edwards, 1 run    
6   NaN     0        NONE  Junaid Siddique [4.0-1-14-0] is back into the ...   
7  19.6   120  over-break                  Dhruv Parashar to Edwards, 1 run    
8   NaN     0        NONE  Scott Edwards, right handed bat, comes to the ...   
9  19.5   119      WICKET   Dhruv Parashar 

In [12]:
df.shape

(20, 9)

In [13]:
##

# END!